In [1]:
!pip install ./python

Processing ./python
    100% |████████████████████████████████| 48.1MB 925kB/s eta 0:00:01   19% |██████▍                         | 9.6MB 26.7MB/s eta 0:00:02    43% |██████████████                  | 21.0MB 29.0MB/s eta 0:00:01
    100% |████████████████████████████████| 6.4MB 6.3MB/s eta 0:00:01
    100% |████████████████████████████████| 8.8MB 6.1MB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 30.6MB/s ta 0:00:01
    100% |████████████████████████████████| 3.1MB 11.8MB/s ta 0:00:01
    100% |████████████████████████████████| 122kB 28.8MB/s ta 0:00:01
    93% |█████████████████████████████▉  | 2.0MB 32.7MB/s eta 0:00:01    100% |████████████████████████████████| 2.2MB 12.8MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 23.9MB/s ta 0:00:01
    100% |████████████████████████████████| 358kB 20.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for unityagents ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-h4h06los/wheels/97/7a/24/09937717b973

In [1]:
from unityagents import UnityEnvironment

import numpy as np
from collections import deque

import torch
import torch.optim as optim

from soccer_model import ActorModel, CriticModel
from memory import Memory

from soccer_agent import Agent
from optimizer import Optimizer


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# environment configuration
env = UnityEnvironment(file_name="Soccer_Windows_x86_64/Soccer.exe", seed=1)

# print the brain names
print(env.brain_names)

# set the goalie brain
g_brain_name = env.brain_names[0]
g_brain = env.brains[g_brain_name]

# set the striker brain
s_brain_name = env.brain_names[1]
s_brain = env.brains[s_brain_name]


# reset the environment
env_info = env.reset(train_mode=True)

# number of agents 
n_goalie_agents = len(env_info[g_brain_name].agents)
print('Number of goalie agents:', n_goalie_agents)
n_striker_agents = len(env_info[s_brain_name].agents)
print('Number of striker agents:', n_striker_agents)

# number of actions
goalie_action_size = g_brain.vector_action_space_size
print('Number of goalie actions:', goalie_action_size)
striker_action_size = s_brain.vector_action_space_size
print('Number of striker actions:', striker_action_size)

# examine the state space 
goalie_states = env_info[g_brain_name].vector_observations
goalie_state_size = goalie_states.shape[1]
print('There are {} goalie agents. Each receives a state with length: {}'.format(goalie_states.shape[0], goalie_state_size))
striker_states = env_info[s_brain_name].vector_observations
striker_state_size = striker_states.shape[1]
print('There are {} striker agents. Each receives a state with length: {}'.format(striker_states.shape[0], striker_state_size))

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 2
        Number of External Brains : 2
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: GoalieBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Unity brain name: StrikerBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 6
        Vector Action descriptions: , , , , , 


['GoalieBrain', 'StrikerBrain']
Number of goalie agents: 2
Number of striker agents: 2
Number of goalie actions: 4
Number of striker actions: 6
There are 2 goalie agents. Each receives a state with length: 336
There are 2 striker agents. Each receives a state with length: 336


In [2]:
# hyperparameters
N_STEP = 8
BATCH_SIZE = 32
GAMMA = 0.995
EPSILON = 0.1
ENTROPY_WEIGHT = 0.001
GRADIENT_CLIP = 0.5
GOALIE_LR = 8e-5
STRIKER_LR = 1e-4


CHECKPOINT_GOALIE_ACTOR = './checkpoint_goalie_actor.pth'
CHECKPOINT_GOALIE_CRITIC = './checkpoint_goalie_critic.pth'
CHECKPOINT_STRIKER_ACTOR = './checkpoint_striker_actor.pth'
CHECKPOINT_STRIKER_CRITIC = './checkpoint_striker_critic.pth'

# Actors and Critics
GOALIE_0_KEY = 0
STRIKER_0_KEY = 0
GOALIE_1_KEY = 1
STRIKER_1_KEY = 1

In [3]:
# NEURAL MODEL
goalie_actor_model = ActorModel( goalie_state_size, goalie_action_size ).to(DEVICE)
goalie_critic_model = CriticModel( goalie_state_size + striker_state_size + goalie_state_size + striker_state_size ).to(DEVICE)
goalie_optim = optim.Adam( list( goalie_actor_model.parameters() ) + list( goalie_critic_model.parameters() ), lr=GOALIE_LR )
# self.optim = optim.RMSprop( list( self.actor_model.parameters() ) + list( self.critic_model.parameters() ), lr=lr, alpha=0.99, eps=1e-5 )


striker_actor_model = ActorModel( striker_state_size, striker_action_size ).to(DEVICE)
striker_critic_model = CriticModel( striker_state_size + goalie_state_size + striker_state_size + goalie_state_size ).to(DEVICE)
striker_optim = optim.Adam( list( striker_actor_model.parameters() ) + list( striker_critic_model.parameters() ), lr=STRIKER_LR )
# self.optim = optim.RMSprop( list( self.actor_model.parameters() ) + list( self.critic_model.parameters() ), lr=lr, alpha=0.99, eps=1e-5 )

goalie_actor_model.load( CHECKPOINT_GOALIE_ACTOR )
goalie_critic_model.load( CHECKPOINT_GOALIE_CRITIC )
striker_actor_model.load( CHECKPOINT_STRIKER_ACTOR )
striker_critic_model.load( CHECKPOINT_STRIKER_CRITIC )


# AGENTS
goalie_0 = Agent( DEVICE, GOALIE_0_KEY, goalie_actor_model, N_STEP )
goalie_optimizer = Optimizer( DEVICE, goalie_actor_model, goalie_critic_model, goalie_optim,  
    N_STEP, BATCH_SIZE, GAMMA, EPSILON, ENTROPY_WEIGHT, GRADIENT_CLIP)

striker_0 = Agent( DEVICE, STRIKER_0_KEY, striker_actor_model, N_STEP )
striker_optimizer = Optimizer( DEVICE, striker_actor_model, striker_critic_model, striker_optim,  
    N_STEP, BATCH_SIZE, GAMMA, EPSILON, ENTROPY_WEIGHT, GRADIENT_CLIP)

goalie_1 = Agent( DEVICE, GOALIE_1_KEY, goalie_actor_model, N_STEP )
goalie_optimizer = Optimizer( DEVICE, goalie_actor_model, goalie_critic_model, goalie_optim,  
    N_STEP, BATCH_SIZE, GAMMA, EPSILON, ENTROPY_WEIGHT, GRADIENT_CLIP)

striker_1 = Agent( DEVICE, STRIKER_1_KEY, striker_actor_model, N_STEP )
striker_optimizer = Optimizer( DEVICE, striker_actor_model, striker_critic_model, striker_optim,  
    N_STEP, BATCH_SIZE, GAMMA, EPSILON, ENTROPY_WEIGHT, GRADIENT_CLIP)

In [4]:
def ppo_train():
    n_episodes = 5000
    team_0_window_score = deque(maxlen=100)
    team_0_window_score_wins = deque(maxlen=100)

    team_1_window_score = deque(maxlen=100)
    team_1_window_score_wins = deque(maxlen=100)

    draws = deque(maxlen=100)

    for episode in range(n_episodes):
        env_info = env.reset(train_mode=True)                        # reset the environment    

        goalies_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
        strikers_states = env_info[s_brain_name].vector_observations # get initial state (strikers)

        goalies_scores = np.zeros(n_goalie_agents)                   # initialize the score (goalies)
        strikers_scores = np.zeros(n_striker_agents)                 # initialize the score (strikers)         

        steps = 0
        
        while True:       
            # select actions and send to environment
            action_goalie_0, log_prob_goalie_0 = goalie_0.act( goalies_states[goalie_0.KEY] )
            action_striker_0, log_prob_striker_0 = striker_0.act( strikers_states[striker_0.KEY] )

            # action_goalie_1, log_prob_goalie_1 = goalie_1.act( goalies_states[goalie_1.KEY] )
            # action_striker_1, log_prob_striker_1 = striker_1.act( strikers_states[striker_1.KEY] )
            
            # random            
            action_goalie_1 = np.asarray( [np.random.choice(goalie_action_size)] )
            action_striker_1 = np.asarray( [np.random.choice(striker_action_size)] )


            actions_goalies = np.array( (action_goalie_0, action_goalie_1) )                                    
            actions_strikers = np.array( (action_striker_0, action_striker_1) )

            actions = dict( zip( [g_brain_name, s_brain_name], [actions_goalies, actions_strikers] ) )

        
            env_info = env.step(actions)                                                
            # get next states
            goalies_next_states = env_info[g_brain_name].vector_observations         
            strikers_next_states = env_info[s_brain_name].vector_observations
            
            # get reward and update scores
            goalies_rewards = env_info[g_brain_name].rewards  
            strikers_rewards = env_info[s_brain_name].rewards
            goalies_scores += goalies_rewards
            strikers_scores += strikers_rewards
                        
            # check if episode finished
            done = np.any(env_info[g_brain_name].local_done)

            # store experiences
            goalie_0_reward = goalies_rewards[goalie_0.KEY]
            goalie_0.step( 
                goalies_states[goalie_0.KEY],
                np.concatenate( 
                    (
                        goalies_states[goalie_0.KEY],
                        strikers_states[striker_0.KEY],
                        goalies_states[GOALIE_1_KEY],
                        strikers_states[STRIKER_1_KEY],
                    ), axis=0 ),
                action_goalie_0,
                log_prob_goalie_0,
                goalie_0_reward 
            )


            striker_0_reward = strikers_rewards[striker_0.KEY]
            striker_0.step(                 
                strikers_states[striker_0.KEY],
                np.concatenate( 
                    (
                        strikers_states[striker_0.KEY],
                        goalies_states[goalie_0.KEY],                        
                        strikers_states[STRIKER_1_KEY],                 
                        goalies_states[GOALIE_1_KEY]                        
                    ), axis=0 ),               
                action_striker_0,
                log_prob_striker_0,
                striker_0_reward
            )


            # exit loop if episode finished
            if done:
                break  

            # roll over states to next time step
            goalies_states = goalies_next_states
            strikers_states = strikers_next_states

            steps += 1

        # learn
        goalie_loss = goalie_optimizer.learn(goalie_0.memory)
        striker_loss = striker_optimizer.learn(striker_0.memory)        

        goalie_actor_model.checkpoint( CHECKPOINT_GOALIE_ACTOR )   
        goalie_critic_model.checkpoint( CHECKPOINT_GOALIE_CRITIC )    
        striker_actor_model.checkpoint( CHECKPOINT_STRIKER_ACTOR )    
        striker_critic_model.checkpoint( CHECKPOINT_STRIKER_CRITIC )

        team_0_score = goalies_scores[goalie_0.KEY] + strikers_scores[striker_0.KEY]
        team_0_window_score.append( team_0_score )
        team_0_window_score_wins.append( 1 if team_0_score > 0 else 0)        

        team_1_score = goalies_scores[GOALIE_1_KEY] + strikers_scores[STRIKER_1_KEY]
        team_1_window_score.append( team_1_score )
        team_1_window_score_wins.append( 1 if team_1_score > 0 else 0 )

        draws.append( team_0_score == team_1_score )
        
        print('Episode: {} \tSteps: \t{} \tGoalie Loss: \t {:.10f} \tStriker Loss: \t {:.10f}'.format( episode + 1, steps, goalie_loss, striker_loss ))
        print('\tRed Wins: \t{} \tScore: \t{:.5f} \tAvg: \t{:.2f}'.format( np.count_nonzero(team_0_window_score_wins), team_0_score, np.sum(team_0_window_score) ))
        print('\tBlue Wins: \t{} \tScore: \t{:.5f} \tAvg: \t{:.2f}'.format( np.count_nonzero(team_1_window_score_wins), team_1_score, np.sum(team_1_window_score) ))
        print('\tDraws: \t{}'.format( np.count_nonzero(draws) ))

        if np.count_nonzero( team_0_window_score_wins ) >= 95:
            break

In [5]:
#ppo_train()

In [6]:
# test the trained agents
team_0_window_score = deque(maxlen=100)
team_0_window_score_wins = deque(maxlen=100)

team_1_window_score = deque(maxlen=100)
team_1_window_score_wins = deque(maxlen=100)

draws = deque(maxlen=100)

for episode in range(20):                                               # play game for n episodes
    env_info = env.reset(train_mode=False)                              # reset the environment    
    goalies_states = env_info[g_brain_name].vector_observations         # get initial state (goalies)
    strikers_states = env_info[s_brain_name].vector_observations        # get initial state (strikers)

    goalies_scores = np.zeros(n_goalie_agents)                          # initialize the score (goalies)
    strikers_scores = np.zeros(n_striker_agents)                        # initialize the score (strikers)

    steps = 0

    while True:
        # select actions and send to environment
        action_goalie_0, log_prob_goalie_0 = goalie_0.act( goalies_states[goalie_0.KEY] )
        action_striker_0, log_prob_striker_0 = striker_0.act( strikers_states[striker_0.KEY] )

        action_goalie_1, log_prob_goalie_1 = goalie_1.act( goalies_states[goalie_1.KEY] )
        action_striker_1, log_prob_striker_1 = striker_1.act( strikers_states[striker_1.KEY] )
        
        # random            
        #action_goalie_1 = np.asarray( [np.random.randint(goalie_action_size)] )
        #action_striker_1 = np.asarray( [np.random.randint(striker_action_size)] )


        actions_goalies = np.array( (action_goalie_0, action_goalie_1) )                                    
        actions_strikers = np.array( (action_striker_0, action_striker_1) )

        actions = dict( zip( [g_brain_name, s_brain_name], [actions_goalies, actions_strikers] ) )

    
        env_info = env.step(actions)                                                
        # get next states
        goalies_next_states = env_info[g_brain_name].vector_observations         
        strikers_next_states = env_info[s_brain_name].vector_observations
        
        # get reward and update scores
        goalies_rewards = env_info[g_brain_name].rewards  
        strikers_rewards = env_info[s_brain_name].rewards
        goalies_scores += goalies_rewards
        strikers_scores += strikers_rewards
                    
        # check if episode finished
        done = np.any(env_info[g_brain_name].local_done)

        # exit loop if episode finished
        if done:
            break  

        # roll over states to next time step
        goalies_states = goalies_next_states
        strikers_states = strikers_next_states

        steps += 1
        
    team_0_score = goalies_scores[goalie_0.KEY] + strikers_scores[striker_0.KEY]
    team_0_window_score.append( team_0_score )
    team_0_window_score_wins.append( 1 if team_0_score > 0 else 0)        

    team_1_score = goalies_scores[GOALIE_1_KEY] + strikers_scores[STRIKER_1_KEY]
    team_1_window_score.append( team_1_score )
    team_1_window_score_wins.append( 1 if team_1_score > 0 else 0 )

    draws.append( team_0_score == team_1_score )
    
    print('Episode {}'.format( episode + 1 ))
    print('\tRed Wins: \t{} \tScore: \t{:.5f} \tAvg: \t{:.2f}'.format( np.count_nonzero(team_0_window_score_wins), team_0_score, np.sum(team_0_window_score) ))
    print('\tBlue Wins: \t{} \tScore: \t{:.5f} \tAvg: \t{:.2f}'.format( np.count_nonzero(team_1_window_score_wins), team_1_score, np.sum(team_1_window_score) ))
    print('\tDraws: \t{}'.format( np.count_nonzero( draws ) ))

env.close()

Episode 1
	Red Wins: 	1 	Score: 	1.10000 	Avg: 	1.10
	Blue Wins: 	0 	Score: 	-1.10000 	Avg: 	-1.10
	Draws: 	0
Episode 2
	Red Wins: 	2 	Score: 	1.10000 	Avg: 	2.20
	Blue Wins: 	0 	Score: 	-1.10000 	Avg: 	-2.20
	Draws: 	0
Episode 3
	Red Wins: 	2 	Score: 	-1.10000 	Avg: 	1.10
	Blue Wins: 	1 	Score: 	1.10000 	Avg: 	-1.10
	Draws: 	0
Episode 4
	Red Wins: 	3 	Score: 	1.10000 	Avg: 	2.20
	Blue Wins: 	1 	Score: 	-1.10000 	Avg: 	-2.20
	Draws: 	0
Episode 5
	Red Wins: 	3 	Score: 	-1.10000 	Avg: 	1.10
	Blue Wins: 	2 	Score: 	1.10000 	Avg: 	-1.10
	Draws: 	0
Episode 6
	Red Wins: 	4 	Score: 	1.10000 	Avg: 	2.20
	Blue Wins: 	2 	Score: 	-1.10000 	Avg: 	-2.20
	Draws: 	0
Episode 7
	Red Wins: 	4 	Score: 	-1.10000 	Avg: 	1.10
	Blue Wins: 	3 	Score: 	1.10000 	Avg: 	-1.10
	Draws: 	0
Episode 8
	Red Wins: 	4 	Score: 	-1.10000 	Avg: 	-0.00
	Blue Wins: 	4 	Score: 	1.10000 	Avg: 	0.00
	Draws: 	0
Episode 9
	Red Wins: 	5 	Score: 	1.10000 	Avg: 	1.10
	Blue Wins: 	4 	Score: 	-1.10000 	Avg: 	-1.10
	Draws: 	0
Episode 10